In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [179]:
class Value:
  def __init__(self, data, _parents=()):
    assert isinstance(data, int) or isinstance(data, float), "Data should be either integer or a float"
    self.data = data
    self._parents = set(_parents)
    self.grad = 0
    self._backward = lambda: None

  def verify_value(self, v):
    if isinstance(v, int) or isinstance(v, float) :
      return Value(v)
    elif isinstance(v, Value):
      return v
    else:
      raise ValueError("Value must either be integer or float")

  def __add__(self, addend):
    addend = self.verify_value(addend)
    res = Value(self.data + addend.data)
    res._parents.update([self, addend])
    def _backward():
      self.grad += res.grad
      addend.grad += res.grad
    res._backward = _backward
    return res

  def __radd__(self, addend):
    return self + addend
  
  def __sub__(self, sh):
    return self + (-sh)
  
  def __mul__(self, multiplier):
    multiplier = self.verify_value(multiplier)
    res = Value(self.data * multiplier.data)
    res._parents.update([self, multiplier])
    def _backward():
      self.grad += multiplier.data * res.grad
      multiplier.grad += self.data * res.grad
    res._backward = _backward
    return res

  def __truediv__(self, divisor):
    return self * (divisor ** -1)

  def __pow__(self, num):
    res = Value(math.pow(self.data, num))
    res._parents.add(self)
    def _backward():
      self.grad += ((num)*math.pow(self.data, num-1)) * res.grad
    res._backward = _backward
    return res

  def tanh(self):
    e = math.exp(2*self.data)
    res = (e - 1)/(e + 1); res = Value(res)
    res._parents.add(self)
    def _backward():
      self.grad += (1 - res.data ** 2) * res.grad
    res._backward = _backward
    return res
  
  def backward(self):
    back_path = []
    visited = set()

    def back_flood(node):
      if node not in visited:
        visited.add(node)
        for parent in node._parents:
          back_flood(parent)
        back_path.append(node)
    back_flood(self)
    self.grad = 1.0
    print("Output gradient set to 1")
    for node in reversed(back_path):
      node._backward()

  def __repr__(self):
    return f"Value(data={self.data})"

In [189]:
import random 

class Neuron:
  def __init__(self, num_inputs):
    self.weights = [Value(random.uniform(-1, 1)) for _ in range(num_inputs)]
    self.bias = Value(random.uniform(-1,1))

  def __call__(self, inputs):
    sum_wx = Value(0.0)
    for inp, weight in zip(inputs, self.weights):
      sum_wx.data += (inp.data * weight.data)
    sum_wx += self.bias
    return sum_wx.tanh()

inputs = [Value(2.0), Value(3.0)]
n = Neuron(2)
n(inputs)


Value(data=-0.9990272512279312)

In [197]:
class Layer:
  def __init__(self, num_inputs, num_out):
    self.neurons = [Neuron(num_inputs) for _ in range(num_out)]

  def __call__(self, x):
    out = [neuron(x) for neuron in self.neurons]
    return out
  
inputs = [Value(2.0), Value(3.0)]
n = Layer(2,3)
n(inputs)

[Value(data=-0.998254862728444),
 Value(data=-0.005235674768969682),
 Value(data=0.15678641702157337)]

In [192]:
# checking tanh and its backward implementation
a = Value(2)
b = Value(-4)
c = Value(6)
d = a * b;
e = d+c;
f = Value(4)
g =  e * f;
h = g.tanh()

In [88]:
h.grad = 1.0
h.backward()

In [90]:
print(h.grad)
for parent in h._parents:
  print(parent, parent.grad)

0
Value(data=-8) 0


In [128]:
x1 = Value(2.0)
x2 = Value(0.0)

w1 = Value(-3.0)
w2 = Value(1.0)

b = Value(6.8813735870195432)

xw1 = x1 * w1
xw2 = x2 * w2
xw1xw2 = xw1 + xw2
p = xw1xw2 + b
q = p.tanh()

In [129]:
q.backward()

Backward called
Gradient set to 1


In [134]:
print(xw1.data, xw1.grad)
print(xw2.data, xw2.grad)

-6.0 0.4999999999999999
0.0 0.4999999999999999


In [135]:
print(x1, x1.grad)
print(x2, x2.grad)
print(w1, w1.grad)
print(w2, w2.grad)

Value(data=2.0) -1.4999999999999996
Value(data=0.0) 0.4999999999999999
Value(data=-3.0) 0.9999999999999998
Value(data=1.0) 0.0


In [175]:
# checking power fuction and its backward implementation

a = Value(-2.0)
b = Value(3.0)
d = a * b
e = a + b
f = d * e
g = f ** 2
g.backward()

Output gradient set to 1


In [178]:
print(g, g.grad)
for parent in g._parents:
  print(parent, parent.grad)

Value(data=36.0) 1.0
Value(data=-6.0) -12.0
